In [2]:
# import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [3]:
# !python -m spacy download en_core_web_sm

# Imports

In [37]:
import csv
import json
import itertools

import pandas as pd
import numpy as np

from nltk.corpus import wordnet as wn
from spacy.lang.en.stop_words import STOP_WORDS
from semantic_text_similarity.models import WebBertSimilarity

import spacy
nlp = spacy.load('en_core_web_sm')

In [38]:
web_model = WebBertSimilarity(batch_size=10)

100%|████████████████████████| 405359924/405359924 [01:30<00:00, 4459113.78B/s]


Failed to download model: web-bert-similarity


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\ikath\\AppData\\Local\\Temp\\tmp0gf0gf5d'

In [ ]:
web_model.predict([("She won an olympic gold medal","The women is an olympic champion")])

In [ ]:
web_model.predict([("anxiety","nervous")])

In [5]:
DATASETS_DIR = "public/datasets/disease and symptoms/"

In [6]:
data = {"users":[]}
with open('DATA.json', 'w') as outfile:
    json.dump(data, outfile)

In [7]:
def write_json(new_data, filename='DATA.json'):
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data["users"].append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)
 

In [8]:
df_tr = pd.read_csv(DATASETS_DIR + 'Training.csv').dropna(how='all', axis=1)

In [9]:
df_tr.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [10]:
df_tr.shape

(4920, 133)

In [11]:
df_tr.iloc[-1]

itching                        0
skin_rash                      1
nodal_skin_eruptions           0
continuous_sneezing            0
shivering                      0
                          ...   
inflammatory_nails             0
blister                        1
red_sore_around_nose           1
yellow_crust_ooze              1
prognosis               Impetigo
Name: 4919, Length: 133, dtype: object

In [12]:
df_tt=pd.read_csv(DATASETS_DIR + 'Testing.csv').dropna(how='all', axis=1)

In [13]:
df_tt.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Allergy
2,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,GERD
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Chronic cholestasis
4,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Drug Reaction


In [14]:
symp = []
disease = []
for i in range(len(df_tr)):
    symp.append(df_tr.columns[df_tr.iloc[i]==1].to_list())
    disease.append(df_tr.iloc[i,-1])

In [15]:
symp[0]

['itching', 'skin_rash', 'nodal_skin_eruptions', 'dischromic _patches']

In [16]:
disease[50]

'Peptic ulcer diseae'

# 1 - get all symtpoms

In [17]:
all_symp_col = list(df_tr.columns[:-1])
def clean_symp(sym):
    return sym.replace('_',' ').replace('.1','').replace('(typhos)','').replace('yellowish','yellow').replace('yellowing','yellow') 

In [18]:
all_symp = [clean_symp(sym) for sym in (all_symp_col)]

In [19]:
# get all symptoms which do not have a synset
ohne_syns = []
mit_syns = []
for sym in all_symp:
    if not wn.synsets(sym):
        ohne_syns.append(sym)
    else:
        mit_syns.append(sym)

In [20]:
len(mit_syns)

33

In [21]:
len(ohne_syns)

99

# 2- preprocess text

In [22]:
def preprocess_sym(doc):
    nlp_doc=nlp(doc)
    d = []
    for token in nlp_doc:
        if not token.text.lower() in STOP_WORDS and token.text.isalpha():
            d.append(token.lemma_.lower() )
    return ' '.join(d)

In [23]:
preprocess_sym("skin peeling")
preprocess_sym("stomach paining")

'stomach pain'

In [24]:
all_symp_pr = [preprocess_sym(sym) for sym in all_symp]

In [25]:
# associates each preprocessed symp with the name of its original column
col_dict = dict(zip(all_symp_pr, all_symp_col))

# 3 - syntactic similarity

In [26]:
def pattern_set(str1, str2):
    list1 = str1.split(' ')
    list2 = str2.split(' ')
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [27]:
# syn similarity with the corpus
def syntactic_similarity(symp_t, corpus):
    most_sim = []
    poss_sym = []

    for symp in corpus:
        d = pattern_set(symp_t, symp)
        most_sim.append(d)

    order = np.argsort(most_sim)[::-1].tolist()

    for i in order:
        if DoesExist(symp_t):
            return 1, [corpus[i]]

        if corpus[i] not in poss_sym and most_sim[i] != 0:
            poss_sym.append(corpus[i])

    if len(poss_sym):
        return 1, poss_sym


    else:
        return 0, None

In [28]:
# Returns all the subsets of this set. This is a generator.
def powerset(seq):
    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item

In [29]:
# Sort list based on length
def sort(a):
    for i in range(len(a)):
        for j in range(i+1,len(a)):
            if len(a[j])>len(a[i]):
                a[i],a[j]=a[j],a[i]
    a.pop()
    return a

In [30]:
# find all permutations of a list
def permutations(s):
    permutations = list(itertools.permutations(s))
    return([' '.join(permutation) for permutation in permutations])

In [31]:
def DoesExist(txt):
    txt=txt.split(' ')
    combinations = [x for x in powerset(txt)]
    sort(combinations)

    for comb in combinations :
        # print(permutations(comb))
        for sym in permutations(comb):
            if sym in all_symp_pr:
                # print(sym)
                return sym
    return False

In [32]:
DoesExist('worried')

False

In [36]:
preprocess_sym('really worried')

'nervous'

In [35]:
syntactic_similarity(preprocess_sym('nervous'), all_symp_pr)

(0, None)

In [ ]:
def check_pattern(inp,dis_list):
    import re
    pred_list=[]
    ptr=0
    patt = "^" + inp + "$"
    regexp = re.compile(inp)
    for item in dis_list:
        if regexp.search(item):
            pred_list.append(item)
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return ptr,None

In [ ]:
check_pattern('nail', all_symp_pr)

# IV- Semantic Similarity

In [ ]:
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
def WSD(word, context):
    sens=lesk(context, word)
    return sens

In [ ]:
def semanticD(doc1,doc2):
    doc1_p=preprocess(doc1).split(' ')
    doc2_p=preprocess_sym(doc2).split(' ')
    score=0
    for tock1 in doc1_p:
        for tock2 in doc2_p:
            syn1 = WSD(tock1,doc1)
            syn2 = WSD(tock2,doc2)
            #syn1=wn.synset(t)
            if syn1 is not None and syn2 is not None :
                x=syn1.wup_similarity(syn2)
                if x is not None and x>0.1:
                    score+=x
    return score/(len(doc1_p)*len(doc2_p))

In [ ]:
semanticD('anxiety', 'nervous')

In [ ]:
syna = wn.synsets('anxiety')
syna[0].definition()

In [ ]:
synn = wn.synsets('nervous')
synn[0].definition()

In [ ]:
synn[0].wup_similarity(syna[0])

In [ ]:
anxiety_synsets = wn.synsets("brittle") 
nervous_synsets = wn.synsets("nervous") 
path = []
wup = []
lch = []

for s1 in anxiety_synsets:
    for s2 in nervous_synsets:
        path.append(s1.path_similarity(s2))
        wup.append(s1.wup_similarity(s2))

pd.DataFrame([path, wup], ["path", "wup"])

In [ ]:
# similarite sem avec ts le corpus
def semantic_similarity(symp_t,corpus):
    max_sim=0
    most_sim=None
    for symp in corpus:
        d=semanticD(symp_t,symp)
        if d>max_sim:
            most_sim = symp
            max_sim = d

    return max_sim, most_sim

In [ ]:
semantic_similarity('nervous', all_symp_pr)

In [ ]:
all_symp_pr.sort()

In [ ]:
all_symp_pr

In [ ]:
from itertools import chain
from nltk.corpus import wordnet
def suggest_syn(sym):
    symp = []
    synonyms = wordnet.synsets(sym)
    lemmas=[word.lemma_names() for word in synonyms]
    lemmas = list(set(chain(*lemmas)))
    for e in lemmas:
        res,sym1=semantic_similarity(e,all_symp_pr)
        if res != 0:
            symp.append(sym1)
    return list(set(symp))

In [ ]:
suggest_syn('worried')

In [ ]:
# recoit client_symptoms et renvoit un dataframe avec 1 pour les symptoms associees
def OHV(cl_sym,all_sym):
    l=np.zeros([1,len(all_sym)])
    for sym in cl_sym:
        l[0,all_sym.index(sym)]=1
    return pd.DataFrame(l, columns =all_symp)

In [ ]:
def contains(small, big):
    a=True
    for i in small:
        if i not in big:
            a=False
    return a

In [ ]:
def possible_diseases(l):
    poss_dis=[]
    for dis in set(disease):
        if contains(l,symVONdisease(df_tr,dis)):
            poss_dis.append(dis)
    return poss_dis

In [ ]:
set(disease)

In [ ]:
#recoit une maladie renvoit tous les sympts
def symVONdisease(df,disease):
    ddf=df[df.prognosis==disease]
    m2 = (ddf == 1).any()
    return m2.index[m2].tolist()
    

In [ ]:
symVONdisease(df_tr,'Jaundice')

# V- Prediction Model (KNN & DT)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [ ]:
X_train=df_tr.iloc[:,:-1]
X_test=df_tt.iloc[:,:-1]
y_train = df_tr.iloc[:,-1]
y_test = df_tt.iloc[:,-1]

In [ ]:
knn_clf=KNeighborsClassifier()
knn_clf.fit(X_train,y_train)

In [ ]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)

In [ ]:
print(classification_report(y_test,knn_clf.predict(X_test)))

In [ ]:
print(classification_report(y_test,dt_clf.predict(X_test)))

##  VI- SEVERITY / DESCRIPTION / PRECAUTION

In [ ]:
severityDictionary=dict()
description_list = dict()
precautionDictionary=dict()

def getDescription():
    global description_list
    with open(DATASETS_DIR + 'symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description={row[0]:row[1]}
            description_list.update(_description)




def getSeverityDict():
    global severityDictionary
    with open(DATASETS_DIR + 'symptom_severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction={row[0]:int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass


def getprecautionDict():
    global precautionDictionary
    with open(DATASETS_DIR + 'symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec={row[0]:[row[1],row[2],row[3],row[4]]}
            precautionDictionary.update(_prec)


In [ ]:
getSeverityDict()
getprecautionDict()
getDescription()

In [ ]:
severityDictionary

In [ ]:
def calc_condition(exp,days):
    sum=0
    for item in exp:
        if item in severityDictionary.keys():
            sum=sum+severityDictionary[item]
    if((sum*days)/(len(exp))>13):
        return 1
        print("You should take the consultation from doctor. ")
    else:
        return 0
        print("It might not be that bad but you should take precautions.")


# Chat

In [ ]:
def getInfo():
    # name=input("Name:")
    print("Your Name \n\t\t\t\t\t\t",end="=>")
    name=input("")
    print("hello ",name)
    return str(name)

In [ ]:
def related_sym(psym1):
    if len(psym1)==1:
        return psym1[0]
    print("searches related to input: ")
    for num,it in enumerate(psym1):
        print(num,")",clean_symp(it))
    if num!=0:
        print(f"Select the one you meant (0 - {num}):  ", end="")
        conf_inp = int(input(""))
    else:
        conf_inp=0

    disease_input=psym1[conf_inp]
    return disease_input

In [ ]:
def main_sp(name,all_symp_col):
    #main Idea: At least two initial sympts to start with
    
    #get the 1st syp ->> process it ->> check_pattern ->>> get the appropriate one (if check_pattern==1 == similar syntaxic symp found)
    print("Enter the main symptom you are experiencing Mr/Ms "+name+"  \n\t\t\t\t\t\t",end="=>")
    sym1 = input("")
    sym1=preprocess_sym(sym1)
    sim1,psym1=syntactic_similarity(sym1,all_symp_pr)
    if sim1==1:
        psym1=related_sym(psym1)
    
    #get the 2nd syp ->> process it ->> check_pattern ->>> get the appropriate one (if check_pattern==1 == similar syntaxic symp found)

    print("Enter a second symptom you are experiencing Mr/Ms "+name+"  \n\t\t\t\t\t\t",end="=>")
    sym2=input("")
    sym2=preprocess_sym(sym2)
    sim2,psym2=syntactic_similarity(sym2,all_symp_pr)
    if sim2==1:
        psym2=related_sym(psym2)
        
    #if check_pattern==0 no similar syntaxic symp1 or symp2 ->> try semantic similarity
    
    if sim1==0 or sim2==0:
        sim1,psym1=semantic_similarity(sym1,all_symp_pr)
        sim2,psym2=semantic_similarity(sym2,all_symp_pr)
        
        #if semantic sim syp1 ==0 (no symp found) ->> suggest possible data symptoms based on all data and input sym synonymes
        if sim1==0:
            sugg=suggest_syn(sym1)
            print('Are you experiencing any ')
            for res in sugg:
                print(res)
                inp=input('')
                if inp=="yes":
                    psym1=res
                    sim1=1
                    break
                
        #if semantic sim syp2 ==0 (no symp found) ->> suggest possible data symptoms based on all data and input sym synonymes
        if sim2==0:
            sugg=suggest_syn(sym2)
            for res in sugg:
                inp=input('Do you feel '+ res+" ?(yes or no) ")
                if inp=="yes":
                    psym2=res
                    sim2=1
                    break
        #if no syntaxic semantic and suggested sym found return None and ask for clarification

        if sim1==0 and sim2==0:
            return None,None
        else:
            # if at least one sym found ->> duplicate it and proceed
            if sim1==0:
                psym1=psym2
            if sim2==0:
                psym2=psym1
    #create patient symp list
    all_sym=[col_dict[psym1],col_dict[psym2]]
    #predict possible diseases
    diseases=possible_diseases(all_sym)
    stop=False
    print("Are you experiencing any ")
    for dis in diseases:
        print(diseases)
        if stop==False:
            for sym in symVONdisease(df_tr,dis):
                if sym not in all_sym:
                    print(clean_symp(sym)+' ?')
                    while True:
                        inp=input("")
                        if(inp=="yes" or inp=="no"):
                            break
                        else:
                            print("provide proper answers i.e. (yes/no) : ",end="")
                    if inp=="yes":
                        all_sym.append(sym)
                        diseases=possible_diseases(all_sym)
                        if len(diseases)==1:
                            stop=True 
    return knn_clf.predict(OHV(all_sym,all_symp_col)),all_sym

    
    

In [ ]:
def chat_sp():
    a=True
    while a:
        name=getInfo()
        result,sym=main_sp(name,all_symp_col)
        if result == None :
            ans3=input("can you specify more what you feel or tap q to stop the conversation")
            if ans3=="q":
                a=False
            else:
                continue

        else:
            print("you may have "+result[0])
            print(description_list[result[0]])
            an=input("how many day do you feel those symptoms ?")
            if calc_condition(sym,int(an))==1:
                print("you should take the consultation from doctor")
            else : 
                print('Take following precautions : ')
                for e in precautionDictionary[result[0]]:
                    print(e)
            print("do you need another medical consultation (yes or no)? ")
            ans=input()
            if ans!="yes":
                a=False
                print("!!!!! thanks for using ower application !!!!!! ")


In [ ]:
df_tr.iloc[-1]

In [ ]:
# import joblib
# knn_clf = joblib.load('model/knn.pkl')  

In [ ]:
symVONdisease(df_tr,"Jaundice")

In [ ]:
knn_clf.predict(OHV(['fatigue', 'weight_loss', 'itching','high_fever'],all_symp_col))

In [ ]:
d=df_tr[df_tr.iloc[:,-1]=="Fungal infection"].sum(axis=0)

In [ ]:
cl=df_tr.columns

In [ ]:
pp=d!=0

In [ ]:
cl[pp]

In [ ]:
d[pp].drop('prognosis')

In [ ]:
chat_sp()